In [ ]:
import pandas as pd
import json

INPUT_FILE = 'terrorism_dataset.csv'
TARGET_MAP = {
    'Military': 'military_police',
    'Police': 'military_police',
    'Government (General)': 'government',
    'Government (Diplomatic)': 'government',
    'Business': 'business',
    'Private Citizens & Property': 'citizens',
    'Transportation': 'transportations'
}
ORDERED_KEYS = ['military_police', 'government', 'business', 'citizens', 'transportations']

def load_and_clean_data(filepath):
    df = pd.read_csv(filepath, encoding='ISO-8859-1', low_memory=False)
    subset_cols = ["iyear", "imonth", "iday", "latitude", "longitude", "targtype1_txt"]
    df = df.dropna(subset=subset_cols)
    df["year"] = df["iyear"].astype(int)
    df["latitude"] = pd.to_numeric(df["latitude"], errors='coerce')
    df["longitude"] = pd.to_numeric(df["longitude"], errors='coerce')
    df['category'] = df['targtype1_txt'].map(TARGET_MAP)
    return df

def generate_top_attacks_json(df):
    local_df = df[df['category'].notna()].copy()
    local_df['nkill'] = pd.to_numeric(local_df['nkill'], errors='coerce').fillna(0)
    local_df['nwound'] = pd.to_numeric(local_df['nwound'], errors='coerce').fillna(0)
    
    output_data = {}
    for key in ORDERED_KEYS:
        cat_df = local_df[local_df['category'] == key].sort_values(by='nkill', ascending=False).head(50)
        output_data[key] = [
            {
                "year": int(row['year']),
                "victims": int(row['nkill'] + row['nwound']),
                "lat": float(row['latitude']),
                "long": float(row['longitude'])
            }
            for _, row in cat_df.iterrows()
        ]
        
    with open('JSON/globe_targets.json', 'w') as f:
        json.dump(output_data, f, indent=2)

clean_df = load_and_clean_data(INPUT_FILE)
generate_top_attacks_json(clean_df)

Loading and cleaning terrorism_dataset.csv...
Generating top_50_attacks_per_category.json...
